# Introduction
State notebook purpose here

### Imports
Import libraries and write settings here.

In [79]:
# Data manipulation
import pandas as pd
import numpy as np
# Options for pandas
pd.options.display.max_columns = 50
pd.options.display.max_rows = 30
# Display all cell outputs
from IPython.core.interactiveshell import InteractiveShell
import datetime
InteractiveShell.ast_node_interactivity = 'all'
from IPython import get_ipython
ipython = get_ipython()
# autoreload extension
if 'autoreload' not in ipython.extension_manager.loaded:
    %load_ext autoreload
%autoreload 2


In [3]:
import tweepy

In [28]:


auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth, wait_on_rate_limit=True)

In [160]:
tweets = []
for tweet in tweepy.Cursor(api.search, "a", geocode="18.2208328,-66.5901489,150km").items(50):
    tweets_properties = {}
    #attributes = ['author', 'contributors','coordinates', 'created_at','destroy','entities','favorite','favorite_count','favorited','geo','id','id_str','in_reply_to_screen_name','in_reply_to_status_id','in_reply_to_status_id_str','in_reply_to_user_id', 
             #         'in_reply_to_user_id_str', 'is_quote_status', 'lang','metadata', 'parse', 'parse_list''place', 'possibly_sensitive',
              #          'quoted_status', 'quoted_status_id','quoted_status_id_str','retweet','retweet_count','retweeted', 'retweets', 'source', 'source_url', 'text',
                #    'truncated','user']
    tweets_properties["id"] = tweet.user.id
    
    tweets_properties["author"] = tweet.user.screen_name
    tweets_properties["created_at"] = tweet.user.created_at

    tweets_properties["location"] = tweet.user.location
    tweets_properties["description"] = tweet.user.description
    tweets_properties["verified"] = tweet.user.verified
    
    tweets_properties["followers"] = tweet.user.followers_count
    tweets_properties["following"] = tweet.user.friends_count
    tweets_properties["favourites_count"] = tweet.user.favourites_count
    
    tweets_properties["statuses_count"] = tweet.user.statuses_count
    tweets_properties["lang"] = tweet.user.lang

    
    today = datetime.datetime.now()
    today = today.replace(hour=23, minute=59, second=59, microsecond=999999) # set from the beggining of the day
    time_to_the_past = 1 # 1 because we want 1 day before today
    yesterday = today - datetime.timedelta(time_to_the_past) 
    
    user = "from:" + str(tweet.user.screen_name)
    # Collecting tweets
    count = 10 # Set the number of tweets to retrieve
    next_day = yesterday + datetime.timedelta(time_to_the_past) # equivalent to today
     
    tuitss = []
    for tuit in tweepy.Cursor(api.search, user,
                               until = next_day.date() # format YYYY-MM-DD in datetime. Not string. Twitter only extract tweets before that date
                               ).items(5):
        tuits = {}
        tuits["id"] = tuit.id
        tuits["created_at"] = tuit.created_at
        tuits["text"] = tuit.text
        print(tuit.text)
        tuits["coordinates"] = tuit.coordinates 
        tuits["geo"] = tuit.geo
        tuits["place"] = tuit.place
        tuits["user"] = tuit.user.screen_name
        tuits["retweet_count"] = tuit.retweet_count
        tuits["favorite_count"] = tuit.favorite_count 
        tuitss.append(tuits)
        

    tweets_properties["tweets"] = pd.DataFrame(tuitss)
    
    
 
    #tweets_properties["tweets"] = api.user_timeline(screen_name = user,count=200)
    tweets.append(tweets_properties)

RT @rafiladiosa: Nunca mezclen los sentimientos con el sexo... se van a joder.
a dormir que mañana se trabaja 😅
RT @_nkshhh: Tengo tik tok no porque hago videos. Sino porque me entretengo viendo lo que suben jajaja 2hrs feliz 🤣🤣🤣🤣🤣
🥴😍 https://t.co/eTviXKOIg7
Esa noche quien la borra?🥴🤤
Es necesario terminar con el negocio en las cárceles https://t.co/2Vq31CZ1BP a través de @ #ElUniversalDigital
Diputados analizan con SISALRIL proyecto busca que ARS devuelvan recursos recibidos durante estado de excepción… https://t.co/HbdWON3V7M
Comisión Ambiental UASD manifiesta hará todo lo posible por conservar la biodiversidad en campus de la academia… https://t.co/m9RQ3gYy7D
El “cambio” o “más de lo mismo'' https://t.co/Axhi61x34D a través de @ #ElUniversalDigital
Pocos hispanos se vacunan contra Covid-19 en NY; periodistas dominicanos exhortan hacerlo https://t.co/OE4sk0IGxJ a… https://t.co/O6UvU0wrHT
RT @juanofalcone: Otro país irán dejando estos sucios a la posteridad. Y otros lo sufrirán.
No f

Me acabo de comer como 10 piezas de pollo frito y 50 arepitas, Dios mio quiero ser flaca pero sin dejar de hartar😭
Bueno mañana comienzo de nuevo, era por carnaval pues.
Yo estaba a dieta pero no aguante!!!!!!
Mi hijo come como 10 veces al día, Dios mio come demasiado y solo tiene casi 3 años.
Mi día a día....
Yo muero con las mariposas 😍😍😍😍😍😍🦋
Este Tatto va a quedarrrrr 😍😍😍😍😍
Necesito a soni 😭😂 y es mi vecina https://t.co/G4litYTJvX
RT @DanishaPacheco: Ya entendiii , tú estás fronteando con una baby que ya yo le di?😂😂😛
Me pongo mala cuando el grooming se tarda en darme a mi bebe 😂😭🐩
RT @NancyFuentesTa: El abdomen mañanero es la única mentira que me gusta.
RT @iTokiOficial: Miedo a que me dejen otra vez llorando a las 4 de la mañana preguntándome que hice mal
Que lástima saber que antes valías la pena y ahora la das.
RT @Tererecafe: Nunca voy a entender para qué un hombre se casa si después va a vivir haciendo "chistes" sobre que odia estar casado y que…
RT @_Yaazmelina: Hay dos tipos 

algo o alguien que me haga bien solo pido
todos igualitos 🥴
RT @textazo: ¿Ya llegaron a la edad en que regañan a sus papás?
RT @lNDlRECTA: ✨ https://t.co/5rCHzZMLNw
RT @aIzxxy: Si lloro enfrente de vos, abrázame porque toqué fondo.
esos que me dicen "me gustaría volver a hablar 🥺" csm que, tuvieron su oportunidad y la desaprovecharon, ni modo
yo todo el tiempo que pasé leyendo mi carta astral: 👁️👄👁️ OMG WEEEEY SISOY, ME IDENTIFICO
RT @amparofrnes: muy mal tienes que portarte conmigo para que no quiera saber nada de ti
@Sat_torres18 JAJAJAJA
estoy hasta la verga de ligar con vatos, o sea, no los celo y se enojan.
Les doy su espacio y se enojan.
Les doy mi… https://t.co/SRCBrZq36I
Psicólogo: plantéate metas a corto plazo.

Yo sin pasaporte, ni plata: irme a Alaska
Ubícate we los Rockstars no contestamos mensajes
RT @dusix13: No prometas un futuro, cuando quieres un instante
@facuusegui Conseguite un perro
Mi personalidad es una verga qué https://t.co/QEAD6Gujod
RT @nochesdecoffee: https:

In [161]:
data = pd.DataFrame(tweets)

In [163]:
data.to_csv('users.csv')

In [162]:
data.head()

,id,author,created_at,location,description,verified,followers,following,favourites_count,statuses_count,lang,tweets
0,1126750275560255488,__BMCC,2019-05-10 07:26:14,,👩🏻‍🏫 |22| 25/10/19 TE AMO TIO 🕊,False,291,227,1636,15041,None,id created_at \ ...
1,976059600,masabala1,2012-11-28 12:55:14,Santo Domingo,Periodista y director del periódico digital ht...,False,2560,2678,3,19914,None,id created_at \ ...
2,1357874332865822721,Belnpal31200801,2021-02-06 02:11:32,,,False,2,27,0,12,None,id created_at \ ...
3,1317932877162680322,Bianca_cajeao,2020-10-18 20:58:07,Merlo (San Luis),No se BRILLA sin OSCURIDAD,False,29,45,246,618,None,id created_at \ ...
4,1336902412062367745,_exequiel_diaz_,2020-12-10 05:16:36,"Tucumán, Argentina Pa'",𝖊𝖘𝖈𝖗𝖎𝖇𝖎𝖊𝖓𝖉𝖔... ...,False,4,45,1013,222,None,id created_at \ ...


In [130]:
data.drop(["description", "tweets"], axis=1, inplace=True)

In [164]:
data.describe()
data.describe().to_csv('describe.csv')

,id,followers,following,favourites_count,statuses_count
count,5.000000e+01,50.000000,50.000000,50.00000,50.000000
mean,7.485169e+17,371.900000,435.740000,9547.50000,15986.820000
std,5.906725e+17,463.526775,573.524862,16616.96035,28684.547502
min,1.357054e+08,2.000000,14.000000,0.00000,12.000000
25%,1.910546e+09,48.750000,82.500000,447.50000,617.250000
50%,1.072996e+18,237.500000,236.000000,2491.50000,4610.500000
75%,1.293566e+18,586.500000,531.750000,8972.00000,14734.500000
max,1.357874e+18,2560.000000,2678.000000,75862.00000,113460.000000


In [131]:
print(data.to_latex(index=False))  

\begin{tabular}{rllllrrrrl}
\toprule
                  id &           author &          created\_at &                 location &  verified &  followers &  following &  favourites\_count &  statuses\_count &  lang \\
\midrule
          3995716426 &       \_graaguiar & 2015-10-19 16:22:27 &   Belo Horizonte, Brasil &     False &        150 &        144 &              2153 &            7768 &  None \\
  945978917420191744 &      VickyWhatts & 2017-12-27 11:25:24 &      Rincón, Puerto Rico &     False &        110 &        193 &              9172 &            5568 &  None \\
           403113439 &     CaRdSuNs1824 & 2011-11-02 01:31:33 &      Prescott Valley, AZ &     False &        151 &        503 &              4324 &            1399 &  None \\
          1064910806 &     elycancelled & 2013-01-06 06:07:11 &      Juncos, Puerto Rico &     False &       3138 &       2232 &            140132 &          178799 &  None \\
           585839238 &          vtjosss & 2012-05-20 15:37:29 &    San

In [13]:
dir(tweets[1])

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_api',
 '_json',
 'author',
 'contributors',
 'coordinates',
 'created_at',
 'destroy',
 'entities',
 'favorite',
 'favorite_count',
 'favorited',
 'geo',
 'id',
 'id_str',
 'in_reply_to_screen_name',
 'in_reply_to_status_id',
 'in_reply_to_status_id_str',
 'in_reply_to_user_id',
 'in_reply_to_user_id_str',
 'is_quote_status',
 'lang',
 'metadata',
 'parse',
 'parse_list',
 'place',
 'possibly_sensitive',
 'quoted_status',
 'quoted_status_id',
 'quoted_status_id_str',
 'retweet',
 'retweet_count',
 'retweeted',
 'retweets',
 'source',
 'source_url',
 'text',
 'truncated',
 'user']

In [166]:
for tweets in data.tweets:
    tweets.to_csv("tweets.csv")
    break
    

In [12]:
tweets_data.head()

,0
0,Status(_api=<tweepy.api.API object at 0x000002...
1,Status(_api=<tweepy.api.API object at 0x000002...
2,Status(_api=<tweepy.api.API object at 0x000002...
3,Status(_api=<tweepy.api.API object at 0x000002...
4,Status(_api=<tweepy.api.API object at 0x000002...


# Analysis/Modeling
Do work here

# Results
Show graphs and stats here

# Conclusions and Next Steps
Summarize findings here

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Introduction" data-toc-modified-id="Introduction-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Introduction</a></span><ul class="toc-item"><li><ul class="toc-item"><li><span><a href="#Imports" data-toc-modified-id="Imports-1.0.1"><span class="toc-item-num">1.0.1&nbsp;&nbsp;</span>Imports</a></span></li></ul></li></ul></li><li><span><a href="#Analysis/Modeling" data-toc-modified-id="Analysis/Modeling-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Analysis/Modeling</a></span></li><li><span><a href="#Results" data-toc-modified-id="Results-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Results</a></span></li><li><span><a href="#Conclusions-and-Next-Steps" data-toc-modified-id="Conclusions-and-Next-Steps-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Conclusions and Next Steps</a></span></li></ul></div>